In [1]:
import numpy as np
import open3d as o3d
import os
import sys


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
office_pcd = o3d.io.read_point_cloud("data\\fragment.ply")
print(office_pcd)

PointCloud with 196133 points.


### Visulization of Office:


In [3]:
o3d.visualization.draw_geometries([office_pcd])

In [4]:
bunny_pcd = o3d.io.read_point_cloud("data\BunnyMesh.ply")
o3d.visualization.draw_geometries([bunny_pcd])

In [19]:
bunny_pcd_np = np.asarray(bunny_pcd.points)

x = bunny_pcd_np[:, 0]
y = bunny_pcd_np[:, 1]
z = bunny_pcd_np[:, 2]

print('X coordinates:', x)
print('Y coordinates:', y)
print('Z coordinates:', z)

print ("Length of x: ", len(x))
print ("Length of y: ", len(y))
print ("Length of z: ", len(z))

X coordinates: [-0.0378297 -0.0447794 -0.0680095 ... -0.0704544 -0.0310262 -0.0400442]
Y coordinates: [0.12794  0.128887 0.151244 ... 0.150585 0.153728 0.15362 ]
Z coordinates: [ 0.00447467  0.00190497  0.0371953  ... -0.0434585  -0.00354608
 -0.00816685]
Length of x:  35947
Length of y:  35947
Length of z:  35947


### Distance based algorithm and visualization:

In [6]:
from sklearn.neighbors import NearestNeighbors

def point_cloud_3d(x, y, z):
    num_points = len(x)  
    k_value = 1 

    points = np.column_stack((x, y, z))

    knn_model = NearestNeighbors(n_neighbors=k_value + 1) 
    knn_model.fit(points)

    distances, indices = knn_model.kneighbors(points)
    average_distance = np.mean(distances[:, 1:])  # Exclude the distance to itself

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np.column_stack((x, y, z)))

    colors = np.zeros((num_points, 3)) 
    above_avg_indices = indices_above_avg(distances, average_distance)
    colors[above_avg_indices] = [1, 0, 0]  # R = 1 so holes point have red color

    pcd.colors = o3d.utility.Vector3dVector(colors)

    o3d.visualization.draw_geometries([pcd])

    print("Average Distance between Data Points:", average_distance)

def indices_above_avg(distances, average_distance):
    return np.where(distances[:, 1:] > average_distance)[0]

point_cloud_3d(x, y, z)


Average Distance between Data Points: 0.0010034659947353545


### Downsampling the point cloud of office room

In [11]:
office_pcd = o3d.io.read_point_cloud("data\\fragment.ply")
print("Downsample the point cloud with a voxel of 0.06")
downpcd_office = office_pcd.voxel_down_sample(voxel_size=0.05)
o3d.visualization.draw_geometries([downpcd_office],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

Downsample the point cloud with a voxel of 0.06


In [20]:
downpcd_office = np.asarray(downpcd_office)

x_office = downpcd_office[:, 0]
y_office = downpcd_office[:, 1]
z_office = downpcd_office[:, 2]

print('X coordinates:', x_office)
print('Y coordinates:', y_office)
print('Z coordinates:', z_office)

print ("Length of x: ", len(x_office))
print ("Length of y: ", len(y_office))
print ("Length of z: ", len(z_office))



X coordinates: [1.75393159 2.65454108 0.99917989 ... 1.29074932 2.05859375 2.109375  ]
Y coordinates: [1.67998798 1.33157701 1.34730215 ... 2.37168017 2.39845955 2.39790979]
Z coordinates: [1.76368447 1.70106854 2.29339578 ... 2.24696573 1.81640625 1.81640625]
Length of x:  4718
Length of y:  4718
Length of z:  4718


In [21]:
### Algorithm checking
point_cloud_3d(x_office, y_office, z_office)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
Average Distance between Data Points: 0.03312844600280005


### Normal Estimation of office point cloud

In [8]:
print("Recompute the normal of the downsampled point cloud")
downpcd_office.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([downpcd_office],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024],
                                  point_show_normal=True)

Recompute the normal of the downsampled point cloud
